In [ ]:
import numpy as np
import pandas as pd
import ast
import re
from openai import OpenAI, BadRequestError
from kiwipiepy.utils import Stopwords # 한국어 자연어 처리 패키지
from kiwipiepy import Kiwi
import faiss

from hard_prompt_e import generate_welcome_message, generate_state_message, classify_prompt, get_chat_prompt, RAG_prompt, matching_rag_prompt, one_agent_rag_prompt
client = OpenAI(api_key=)

In [3]:
class base_f:
    def __init__(self, api_key, file_name):
        self.api_key = api_key
        self.client = OpenAI(api_key=api_key)
        self.df = pd.read_pickle(file_name)
        self.Kiwi = Kiwi(typos="basic", model_type='sbg')
        self.stopwords = Stopwords().remove(('사람', 'NNG'))

    # 와/과 구분    
    def ends_with_jong(self, kstr):
        k = kstr[-1]
        if "가" <= k <= "힣":
            if (ord(k)-ord("가")) % 28 > 0:
                return "과"
            else:
                return "와"
        else:
            return "와"
        
    # 메시지의 의도를 파악하는 함수
    def classify(self, msg, matched_agent):
        # 활동명만 입력했을 경우 Match나 Buy&Sell로 연결되는 것을 방지하기 위함
        if msg.replace(" ", "") in matched_agent:
            return ["Chat"]
        else:
            prompt = classify_prompt(msg)
            response = self.client.chat.completions.create(
            model = "gpt-4-1106-preview",
            messages=[{"role": "system", "content": prompt}],
            temperature = 0
            )
            response = response.choices[0].message.content
            try:
                classification = ast.literal_eval(response)
                if classification[0] == "Match":
                    return ["Match", classification[1]]
                else:
                    return ["Chat"]
            except:
                return ["Chat"]
    
    # 형태소로 분리, 명사 추출
    def tokenize_N(self, text):
        split_s = self.Kiwi.tokenize(text, stopwords=self.stopwords, normalize_coda=True)
        N_list = [i.form for i in split_s if i.tag == "NNG" or i.tag == "NNP"]
        split_list = [i.form for i in split_s]
        split_f = ','.join(split_list).replace(",", " ")
        return split_f, N_list

base_f = base_f(api_key, "20240222_JJAgentData.pkl")

In [4]:
class RAG_f:
    def __init__(self):
        self.client = base_f.client
        self.df =  base_f.df
        self.tokenize_N = base_f.tokenize_N
        self.vectors_list = self.df['Embeddings'].tolist()
        self.index = self.faiss_index(self.vectors_list)
    
    # 텍스트 벡터값으로 변환 (리스트 형태로)
    def get_embedding(self, text, model="text-embedding-3-large"):
        text = str(text).replace("\n", " ")
        return self.client.embeddings.create(input=[text], model=model).data[0].embedding
    
    # 텍스트 벡터값으로 변환 (array 형식으로 맞추기)
    def to_vector(self, x):
        _vector_ = self.get_embedding(x)
        _vector = np.array([_vector_]).astype(np.float32)
        return _vector

    # 벡터끼리의 정보를 계산해두고 인덱스 지정
    def faiss_index(self, vectors_list):
        l = [len(v) for v in vectors_list]
        vectors = np.array(vectors_list).astype(np.float32)
        self.index = faiss.IndexFlatL2(max(l))
        self.index.add(vectors)
        return self.index

    # keyword가 들어간 문장을 2개 검색
    def keyword_search(self, keyword):
        final_list = []
        k_search = self.df[self.df['메시지'].str.contains(keyword)].index
        if len(k_search) > 1:
            n = 0
            while len(final_list) < 2:
                final_list += [(self.df.iloc[k_search[n], 0], self.df.iloc[k_search[n], 1])]
                n += 1
        elif len(k_search) == 1:
            final_list += [(self.df.iloc[k_search[0], 0], self.df.iloc[k_search[0], 1])]
        else:
            pass
        return final_list

    # 입력 문장과 유사한 문장들 검색
    def extract_sentences(self, indices, exclude_nickname=None):
        extracted_info = []
        for idx in indices.flatten():
            if idx < len(self.df):
                agent_name = self.df.iloc[idx]['Agent']
                message = self.df.iloc[idx]['메시지']
                # 사용자의 nickname을 포함하는 항목을 제외
                if exclude_nickname is None or agent_name != exclude_nickname:
                    extracted_info.append((agent_name, message))
        return extracted_info

    # 기존 list 내에 있는 문장을 제외하고 추가로 검색
    # k1은 처음 검색해올 문장 개수, k2는 최종적으로 출력할 문장 개수(중복된 문장이 몇개인지에 따라 달라짐)
    def search_vector(self, sentence, k1, k2, dp_list):
        list_ = []
        _, indices = self.index.search(self.to_vector(sentence), k=k1)
        extracted_info = self.extract_sentences(indices)
        for info in extracted_info[:k2]:
            if info not in dp_list:
                list_.append(info)
        return list_

    def find_closest_match(self, msg_object, user_input, number):
        final_list = []
        try:
            tokenized_input, n_list = self.tokenize_N(user_input)
            n_list = n_list[:number[0]]
            for n_word in n_list:
                final_list += self.keyword_search(n_word)
            final_list = list(set(final_list))
            # final_list.append('---------------keyword------------------')
            if msg_object == '':
                pass
            else:
                final_list += self.search_vector(msg_object, number[1]*2, number[1], final_list)
            # final_list.append('---------------object------------------')
            final_list += self.search_vector(tokenized_input, number[2]*2, number[2], final_list)
            # final_list.append('---------------tokenized----------------')
            _, indices = self.index.search(self.to_vector(user_input), k=number[3])
            extracted_info = self.extract_sentences(indices)
            n = 0
            while len(final_list) < number[3]:
                if extracted_info[n] not in final_list:
                    final_list.append(extracted_info[n])
                n += 1
        except BadRequestError:
            _, indices = self.index.search(self.to_vector(user_input), k=number[3])
            final_list = self.extract_sentences(indices)
        except IndexError:
            final_list = [(x, y) for x, y in zip(base_f.df['Agent'], base_f.df['메시지'])]
        return final_list

RAG_f = RAG_f()

In [13]:
class JarvisJust:
    def __init__(self, number, print_method):
        self.client = base_f.client
        self.number = number
        self.method = print_method

    # 의도판단 + RAG
    def make_profile(self, msg, matched_agent):
        classification= base_f.classify(msg, matched_agent)
        if classification[0] == "Chat":
            return "Chat"
        else:
            profile = RAG_f.find_closest_match(classification[1], msg, self.number)
            return profile
    
    def chat(self, msg, state):
        chat_prompt = get_chat_prompt(msg)
        state.append({"role": "system", "content": chat_prompt})
        response = base_f.client.chat.completions.create(
                        model="gpt-4-1106-preview",
                        messages=state,
                        stream=False)
        answer = response.choices[0].message.content
        del state[-1]
        state.append({"role": "assistant", "content": answer})
        return answer, state

    def print_1(self, profile, msg, state):
        state.append({"role": "system", "content": RAG_prompt + f" 사용자: <{msg}>. 매칭리스트: {profile}"})
        response = base_f.client.chat.completions.create(
                        model="gpt-4-1106-preview",
                        messages=state,
                        stream=False)
        answer = response.choices[0].message.content
        del state[-1]
        state.append({"role": "system", "content": f"매칭리스트: {profile}"})
        state.append({"role": "assistant", "content": answer})
        return answer, state
    
    def print_2(self, profile, msg, state):
        def find_agent(msg, profile): 
            RAG_prompt = matching_rag_prompt(msg, profile)
            response = base_f.client.chat.completions.create(
                            model = "gpt-4-1106-preview",
                            messages=[{"role": "system", "content": RAG_prompt}],
                            temperature = 0
                        )
            response = response.choices[0].message.content
            return response
        one_agent = find_agent(msg, profile)
        for i in profile:
            if i[0] == one_agent:
                one_agent_msg = i[1]
        
        filtered_prompt = one_agent_rag_prompt(one_agent, one_agent_msg)
        state.append({"role": "system", "content": filtered_prompt})
        response = base_f.client.chat.completions.create(
                        model="gpt-4-1106-preview",
                        messages=state,
                        stream=False)
        answer = response.choices[0].message.content
        del state[-1]
        state.append({"role": "system", "content": f"매칭리스트: {profile}"})
        state.append({"role": "assistant", "content": answer})
        return answer, state
    
    def final_print(self, profile, msg, state, print_method=None):
        if print_method is None:
            print_method = self.method
        if print_method == 1:
            return self.print_1(profile, msg, state)
        else:
            return self.print_2(profile, msg, state)

    def message_process(self, msg, state, matched_agent):
        state.append({"role": "user", "content": msg})
        if not msg:
            empty_msg = '대화를 입력해주세요.'
            state.append({"role": "assistant", "content": empty_msg})
            return "end", empty_msg, state
        for i in range(len(matched_agent)):
            if matched_agent[i] in msg.replace(" ", ""):
                if "연결" in msg or "매칭" in msg:
                    jong = base_f.ends_with_jong(matched_agent[i])
                    answer = f"{matched_agent[i]+jong} 연결해드리겠습니다."
                    return "end", answer, state
                else:
                    continue
        return "next", "_", state
        
    def final_process(self, state, matched_agent, matched_msg, user_name):
        sub_matched_agent = re.findall(r'Agent: (.*?)\n', state[-1]['content'])
        for i in range(len(sub_matched_agent)):
            matched_agent.append(sub_matched_agent[i])
        sub_matched_msg = re.findall(r'메시지: (.*?)\n', state[-1]['content'])
        for i in range(len(sub_matched_msg)):
            matched_msg.append(sub_matched_msg[i])
        matched_agent = matched_agent[-14:]
        matched_msg = matched_msg[-14:]
        del state[0]
        state = state[-9:]
        state.insert(0, generate_state_message(user_name))
        return state, matched_agent, matched_msg

JarvisJust = JarvisJust([2, 2, 2, 10], 2)

In [14]:
def go_JJ(user_name):
    state = [generate_state_message(user_name)]
    matched_agent = []
    matched_msg = []
    print(generate_welcome_message(user_name))
    while True:
        msg = input("사용자: ")
        print('사용자: '+msg)
        if msg == 'break':
            break
        route, answer, state = JarvisJust.message_process(msg, state, matched_agent)
        if route == "next":
            profile = JarvisJust.make_profile(msg, matched_agent)
            if profile == "Chat":
                answer, state = JarvisJust.chat(msg, state)
            else:
                answer, state = JarvisJust.final_print(profile, msg, state)
        state, matched_agent, matched_msg = JarvisJust.final_process(state, matched_agent, matched_msg, user_name)
        print(answer)
    return matched_agent, state

In [15]:
go_JJ("윤이지")

안녕하세요? 윤이지님. 
                        저는 JarvisJust(JJ)입니다. 무엇을 도와드릴까요? 

                        찾으시는 내용을 아래처럼 입력해주시면 JJ가 연결해드립니다.
                        예1) 진미채 살 수 있는 곳을 찾아줘
                        예2) 주말에 축구 같이할 사람 찾아줘

                        연결된 이후 상품 구매를 희망하시면,                                        
                        ‘닉네임과 상품 결제해줘’를
                        예) OOO(닉네임) 상품 결제해줘

                        1:1채팅 연결을 원하시면, 
                        ‘닉네임과 연결해줘’를 입력해주세요.
                        예) OOO(닉네임) 연결해줘
사용자: 진미채 사고싶어
사용자의 요구사항을 바탕으로 다음 에이전트를 추천합니다.

Agent: 반찬의달인
메시지: 맛있고 간편한 진미채 볶음을 찾고 계신가요? 주방의 신선한 반찬이자 술안주로 제격인 백진미와 홍진미 오징어채 400g을 만나보세요. 집에서든 친구들과의 모임에서든 완벽하게 어울리는 이 맛있는 진미채 볶음으로 간편하면서도 풍미 가득한 식사를 즐기실 수 있습니다. 지금 바로 중부시장(서울특별시 중구 을지로30길 29)에서 만나보세요!
매칭 이유: 반찬의달인은 맛있고 간편한 진미채를 제공합니다.
사용자: 주말에 축구 같이 할 사람 찾아줘
사용자의 요구사항을 바탕으로 다음 에이전트를 추천합니다.

Agent: 축구사랑 주말전사
메시지: 주말에 축구 팀을 구성하려고 해요. 실력보다는 즐기는 것에 중점을 두고 싶습니다. 함께 운동하며 즐거운 시간을 보내고 싶어요!
매칭 이유: 주말에 축구를 즐기고자 하는 분위기가 잘 맞습니다.
사용자: br

(['반찬의달인', '축구사랑 주말전사'],
 [{'role': 'system',
   'content': "You are a matchmaker, And the user's name is 윤이지.\n                 Remember this, Refer to the chatting history below, and Answer the user's message in Korean."},
  {'role': 'user', 'content': '진미채 사고싶어'},
  {'role': 'system',
   'content': "매칭리스트: [('한정판홍진형님', '선착순 300명에게만 특별한 기회를 드리는 한정판 홍진미 400g을 만나보세요! 이 맛있는 기회를 놓치지 마시고, 중부시장(서울특별시 중구 을지로30길 29)에서 여러분을 기다리고 있습니다.'), ('반찬의달인', '맛있고 간편한 진미채 볶음을 찾고 계신가요? 주방의 신선한 반찬이자 술안주로 제격인 백진미와 홍진미 오징어채 400g을 만나보세요. 집에서든 친구들과의 모임에서든 완벽하게 어울리는 이 맛있는 진미채 볶음으로 간편하면서도 풍미 가득한 식사를 즐기실 수 있습니다. 지금 바로 중부시장(서울특별시 중구 을지로30길 29)에서 만나보세요!'), ('특볶은멸치소개꾼', '안녕하세요! 저는 특볶음멸치 1.5kg를 구매하고 싶어요. 혹시 구매 가능한 곳을 연결해주실 수 있을까요?'), ('황태채사장', '황태채 제품을 구매하고 싶습니다.'), ('곱창마스터', '저는 무가미 곱창구운김을 구매해보고 싶습니다.'), ('멸치대장', '안녕하세요, 지금 지리멸치 1.5kg 구매하고 싶어서 연락드렸어요. 혹시 재고가 있을까요?'), ('국물요정멸치매니아', '국물멸치를 구매해보고 싶습니다.'), ('황태마스터', '저는 진부령 용대리 황태채 제품을 구입하고 싶습니다.'), ('황태마스터', '안녕하세요, 저는 진부령에서 나는 용대리 황태채를 구매하고 싶어서 연락드렸어요. 혹시 좋은 품질의 황태채를 